<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Raw-Data" data-toc-modified-id="Raw-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Raw Data</a></span></li><li><span><a href="#Split" data-toc-modified-id="Split-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Split</a></span></li></ul></div>

In [1]:
import sys
import os
import time
import datetime

import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

---

## Raw Data

In [15]:
data_folder = "/Users/xu.zhu/Desktop/Data/price_adjustment/th_202103"
data_filepath = os.path.join(data_folder, "RawData_ToConfirm.csv")

df = pd.read_csv(data_filepath, encoding="utf-8-sig")
df.shape
df.head(3)

(1515304, 15)

,grass_region,gp_account_owner,gp_account_name,seller_name,level1_category,level2_category,shop_id,item_id,model_id,item_weight_g,current_listing_price,current_hidden_fee,new_hidden_fee,new_listing_price,price_issue
0,TH,Asher Yu,深圳市三阳新能源科技有限公司,mikihouse.th,Fashion Accessories,Hair Accessories,219528671,3004800539,53728073313,50.0,45.0,10,5,40.0,0
1,TH,Asher Yu,深圳市三阳新能源科技有限公司,mikihouse.th,Fashion Accessories,Hair Accessories,219528671,3004800539,53728073314,50.0,45.0,10,5,40.0,0
2,TH,Asher Yu,深圳市三阳新能源科技有限公司,mikihouse.th,Fashion Accessories,Hair Accessories,219528671,3004800539,53728073315,50.0,38.0,10,5,33.0,0


In [16]:
issue_df = df[df["price_issue"]==1]
issue_df.shape[0]

issue_df.to_csv(os.path.join(data_folder, "RawData_issue.csv"), index=False, encoding="utf-8-sig")

20589

In [17]:
raw_df = df[df["price_issue"]==0]
raw_df.shape
raw_df.tail(2)

(1494715, 15)

,grass_region,gp_account_owner,gp_account_name,seller_name,level1_category,level2_category,shop_id,item_id,model_id,item_weight_g,current_listing_price,current_hidden_fee,new_hidden_fee,new_listing_price,price_issue
1515302,TH,Zoe Zhou,南京铂彩创意珠宝设计有限公司,platinumcolor.th,Watches & Glasses,Glasses,237658759,9601940448,42259709251,80.0,119.0,16,8,111.0,0
1515303,TH,Zoe Zhou,南京铂彩创意珠宝设计有限公司,platinumcolor.th,Watches & Glasses,Glasses,237658759,9601940448,42259709252,80.0,119.0,16,8,111.0,0


In [19]:
raw_df.columns.to_list()

gp_cnt = len(set(raw_df["gp_account_name"].to_list()))
shop_cnt = len(set(raw_df["shop_id"].to_list()))
sku_cnt = len(set(raw_df["item_id"].to_list()))
model_cnt = len(set(raw_df["model_id"].to_list()))
print("GP: {0}\nShop: {1}\nSKU: {2}\nModel: {3}".format(gp_cnt, shop_cnt, sku_cnt, model_cnt))

['grass_region',
 'gp_account_owner',
 'gp_account_name',
 'seller_name',
 'level1_category',
 'level2_category',
 'shop_id',
 'item_id',
 'model_id',
 'item_weight_g',
 'current_listing_price',
 'current_hidden_fee',
 'new_hidden_fee',
 'new_listing_price',
 'price_issue']

GP: 112
Shop: 133
SKU: 144956
Model: 1494715


## Split

In [14]:
def create_new_folder(folder_path):
    if os.path.exists(folder_path) == False:
        os.mkdir(folder_path)
    else:
        pass

In [ ]:
key_cols = ["gp_account_owner", "gp_account_name", "shop_id", "item_id", "model_id", "new_listing_price"]

In [50]:
to_confirm_folder = "/Users/xu.zhu/Desktop/Data/price_adjustment/th_202103/to_confirm"
create_new_folder(to_confirm_folder)

raw_df = raw_df.drop(columns=["price_issue"])
raw_cols = raw_df.columns.to_list()
raw_cols_cnt = len(raw_cols)
index_dict = {
    1: "A", 2: "B", 3: "C", 4: "D", 5: "E", 6: "F", 7: "G",
    8: "H", 9: "I", 10: "J", 11: "K", 12: "L", 13: "M", 14: "N",
    15: "O", 16: "P", 17: "Q", 18: "R", 19: "S", 20: "T", 21: "U",
    22: "V", 23: "W", 24: "X", 25: "Y", 26: "Z"
}

row_cutoff = 500000

gp_owners = list(set(raw_df["gp_account_owner"]))
for owner in gp_owners:
    owner_folder = os.path.join(to_confirm_folder, owner)
    create_new_folder(owner_folder)
    
    owner_df = raw_df[raw_df["gp_account_owner"] == owner]
    gp_accounts = list(set(owner_df["gp_account_name"]))
    for gp_account in gp_accounts:
        account_df = owner_df[owner_df["gp_account_name"] == gp_account]
        account_df["confirmed"] = None
        additional_col_index = index_dict[raw_cols_cnt + 1]
        
        gp_row_cnt = account_df.shape[0]
        if gp_row_cnt < row_cutoff:
            excel_writer = pd.ExcelWriter(
                    os.path.join(owner_folder, "{0}.xlsx".format(gp_account)),
                    engine="xlsxwriter"
                )
            account_df.to_excel(
                excel_writer,
                sheet_name="gp_account",
                index=False,
                encoding="utf-8-sig"
            )
            worksheet = excel_writer.sheets["gp_account"]
            worksheet.data_validation(
                "{index}2:{index}{row_num}".format(index=additional_col_index, row_num=gp_row_cnt+1),
                {"validate": "list", "source": ["Y", "N"]}
            )
            excel_writer.save()
            
        else:
            print("- Big DataFrame: {0}, Row Number: {1}".format(gp_account, gp_row_cnt))
            split_count=1
            start_index = 0
            for i in range(gp_row_cnt//row_cutoff + 1):
                split_account_df = account_df.iloc[start_index:start_index+row_cutoff]
                
                excel_writer = pd.ExcelWriter(
                    os.path.join(owner_folder, "{0}_{1}.xlsx".format(gp_account, split_count)),
                    engine="xlsxwriter"
                )
                split_account_df.to_excel(
                    excel_writer,
                    sheet_name="gp_account",
                    index=False,
                    encoding="utf-8-sig"
                )
                worksheet = excel_writer.sheets["gp_account"]
                worksheet.data_validation(
                    "{index}2:{index}{row_num}".format(index=additional_col_index, row_num=gp_row_cnt+1),
                    {"validate": "list", "source": ["Y", "N"]}
                )
                excel_writer.save()
                
                start_index += row_cutoff
                split_count += 1